# Climate Change Trend Analysis Using Time-Series Data
This notebook demonstrates synthetic climate data generation, visualization, time-series decomposition, and forecasting using ARIMA.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pathlib import Path

In [ ]:
# Generate synthetic time-series data
np.random.seed(42)
n_months = 240  # 20 years monthly data
dates = pd.date_range(start='2000-01-01', periods=n_months, freq='M')

# Temperature anomaly
temp_trend = np.linspace(0, 1.2, n_months)
temp_season = 0.2 * np.sin(2 * np.pi * np.arange(n_months) / 12)
temp_noise = np.random.normal(0, 0.05, n_months)
temperature_anomaly = temp_trend + temp_season + temp_noise

# CO2 concentration
co2_trend = 370 + np.linspace(0, 50, n_months)
co2_season = 3 * np.sin(2 * np.pi * np.arange(n_months) / 12)
co2_noise = np.random.normal(0, 1, n_months)
co2_concentration = co2_trend + co2_season + co2_noise

# Sea level rise
sea_level_trend = 0.01 * (np.arange(n_months) ** 1.1)
sea_level_noise = np.random.normal(0, 0.5, n_months)
sea_level_rise = sea_level_trend + sea_level_noise

# DataFrame
df = pd.DataFrame({
    'date': dates,
    'temperature_anomaly': temperature_anomaly,
    'co2_concentration': co2_concentration,
    'sea_level_rise': sea_level_rise
}).set_index('date')

# Save dataset
df.to_csv('climate_change_time_series.csv')
df.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index, df['temperature_anomaly'], label='Temperature Anomaly (°C)')
plt.plot(df.index, df['co2_concentration']/100, label='CO₂ Concentration (/100 ppm)')
plt.plot(df.index, df['sea_level_rise']/10, label='Sea Level Rise (/10 cm)')
plt.title('Synthetic Climate Change Indicators (2000–2019)')
plt.xlabel('Year')
plt.ylabel('Scaled values')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
decomp = seasonal_decompose(df['temperature_anomaly'], model='additive', period=12)
decomp.plot()
plt.suptitle('Seasonal Decomposition of Temperature Anomaly', y=1.02)
plt.show()

In [ ]:
# Train-test split
train_size = int(len(df) * 0.8)
train, test = df['temperature_anomaly'][:train_size], df['temperature_anomaly'][train_size:]

# ARIMA model
model = ARIMA(train, order=(2, 1, 2))
fit = model.fit()
forecast = fit.forecast(steps=len(test))

# Evaluation
mae = mean_absolute_error(test, forecast)
rmse = mean_squared_error(test, forecast, squared=False)
print(f'Forecast MAE: {mae:.3f}')
print(f'Forecast RMSE: {rmse:.3f}')

# Plot
plt.figure(figsize=(10,5))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test', color='orange')
plt.plot(test.index, forecast, label='Forecast', color='red', linestyle='--')
plt.title('ARIMA Forecast of Temperature Anomaly')
plt.xlabel('Year')
plt.ylabel('Temperature Anomaly (°C)')
plt.legend()
plt.tight_layout()
plt.show()